# Tensor

The tensor protocol lets you query the tensor (vector, matrix, etc.) representation of a bloq. For example, we can easily inspect the familiar unitary matrix representing the controlled-not operation:

In [ ]:
from qualtran.bloqs.basic_gates import CNOT

cnot = CNOT()
cnot.tensor_contract().real

Bloqs can represent states, effects, and non-unitary operations. Below, we see the vector representation of the plus state and zero effect.

In [ ]:
from qualtran.bloqs.basic_gates import PlusState, ZeroEffect

print('|+> \t', PlusState().tensor_contract())  # state
print('<0| \t', ZeroEffect().tensor_contract()) # effect

We can also look at the non-unitary `And` operation which outputs its result to a new qubit. As such, it's shape is $(2^3, 2^2)$ instead of being a square matrix.

In [ ]:
from qualtran.bloqs.mcmt import And

And().tensor_contract().shape

## Interface

The main way of accessing the dense, contracted, tensor representation of a bloq or composite bloq is through the `Bloq.tensor_contract()` method as we've seen.

All functionality for the tensor protocol is contained in the `qualtran.simulation.tensor` module. For example: `Bloq.tensor_contract()` is an alias for `bloq_to_dense(bloq: Bloq)` within that module. 

In [ ]:
import numpy as np
from qualtran.simulation.tensor import bloq_to_dense

np.array_equal(
    cnot.tensor_contract(),
    bloq_to_dense(cnot)
)

## Additional functionality

A composite bloq has a 1-to-1 mapping with a tensor network. We use [Quimb](https://quimb.readthedocs.io/) to handle efficient contraction of such networks.

The most important library function is `qualtran.simulation.tensor.cbloq_to_quimb`. This will build a quimb `qtn.TensorNetwork` tensor network representation of the composite bloq. You may want to manipulate this object directly using the full Quimb API. Otherwise, this function is used as the workhorse behind the public functions and methods like `Bloq.tensor_contract()`. 

As an example below, we decompose `MultiAnd` into a `CompositeBloq` consisting of a ladder of two-bit `And`s.

In [ ]:
from qualtran.bloqs.mcmt import MultiAnd
from qualtran.drawing import show_bloq

bloq = MultiAnd(cvs=(1,)*4)
cbloq = bloq.decompose_bloq()
show_bloq(cbloq)

This composite bloq graph can be transformed into a quimb tensor network. Some of the visual flair has been lost, but the topology of the graph is the same.

In [ ]:
from qualtran.simulation.tensor import cbloq_to_quimb

tn, fix = cbloq_to_quimb(cbloq)
tn.draw(show_inds=False)

The entire suite of Quimb tools are now available.

In [ ]:
tn.contraction_info()

## Implementation

The `qualtran.simulation.tensor` functions rely on the `Bloq.add_my_tensors(...)` method to implement the protocol. This is where a bloq's tensor information is actually encoded.

Bloq authors may want to override this method. The library will provide a (partial) `qtn.TensorNetwork` as well as dictionaries of incoming and outgoing indices (keyed by register name) to asist the author in matching up dimensions of their `np.ndarray` to the incoming and outgoing wires. Bloq authors are encouraged to read the docstring for this method for more details.

Below, we write our own `CNOT` bloq with custom tensors.

In [ ]:
from functools import cached_property
from typing import Any, Dict, Tuple

import numpy as np
import quimb.tensor as qtn
from attrs import frozen

from qualtran import Bloq, Signature, Soquet, SoquetT, Register, Side

@frozen
class MyCNOT(Bloq):
    @cached_property
    def signature(self) -> 'Signature':
        return Signature.build(ctrl=1, target=1)

    def add_my_tensors(
        self, tn: qtn.TensorNetwork, tag: Any,
        *, incoming: Dict[str, SoquetT], outgoing: Dict[str, SoquetT],
    ):
        # The familiar CNOT matrix. We make sure to
        # cast this to np.complex128 so we don't accidentally
        # lose precision anywhere else in the contraction.
        matrix = np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 0, 1],
            [0, 0, 1, 0],
        ], dtype=np.complex128)
        
        # According to our signature, we have two thru-registers.
        # This means two incoming and two outgoing wires.
        # We'll reshape our matrix into the more natural n-dimensional
        # tensor form.
        tensor = matrix.reshape((2,2,2,2))
        

        tn.add(qtn.Tensor(
            data=tensor, 
            inds=(outgoing['ctrl'], outgoing['target'], 
                  incoming['ctrl'], incoming['target']),
            tags = ['cnot', tag],
        ))

In [ ]:
# Sanity check
MyCNOT().tensor_contract()

## Default Fallback

If a bloq does not override `add_my_tensors(...)`, the default fallback will be used by `qualtran` to support the tensor protocol.

By default, qualtran will fall back on the tensor contraction of the decomposition. This recursion will continue until each leaf bloq defines `add_my_tensors` or a bloq cannot be further decomposed.

Specifically, the system will:

 - decompose the bloq with `bloq.decompose_bloq()` into a composite bloq.
 - use the result of `cbloq_as_contracted_tensor(...)` as the bloq's tensor.
 
 
For example, below we author a `BellState` bloq. We define a decomposition but do not explicitly provide tensor information.

In [ ]:
from qualtran import QBit
from qualtran.bloqs.basic_gates import PlusState, ZeroState

@frozen
class BellState(Bloq):
    @cached_property
    def signature(self) -> 'Signature':
        return Signature([
            Register('q0', QBit(), side=Side.RIGHT),
            Register('q1', QBit(), side=Side.RIGHT)
        ])

    def build_composite_bloq(self, bb):
        q0 = bb.add(PlusState())
        q1 = bb.add(ZeroState())

        q0, q1 = bb.add(CNOT(), ctrl=q0, target=q1)
        return {'q0': q0, 'q1': q1}


Nevertheless, the system can recursively determine the tensor form:

In [ ]:
print(BellState().tensor_contract())

Note that the composite bloq is fully contracted to a dense tensor at each level of decomposition, which likely will prevent quimb from finding the best contraction ordering. See `flatten_for_tensor_contraction` if this is an issue.